## chunking과 vector DB를 활용해 RAG구조를 구성하고 커스터마이징 된 대화형 AI를 구현해보자

In [1]:
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
# langchain과 openai가 연동된 라이브러리에서 pre-trained Embedding 모델 임포트
from langchain_openai import OpenAIEmbeddings

### 1. gpt-3.5-turbo 모델에게 일반적인 질의

In [2]:
MY_API_KEY = getpass.getpass("OpenAI API Key :")

OpenAI API Key : ········


In [3]:
client = OpenAI(api_key=MY_API_KEY)

In [5]:
completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':'기술평가가 뭐야?'}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

기술평가는 특정 기술이나 제품의 성능, 품질, 안전성, 신뢰성 등을 평가하는 과정을 말합니다. 이를 통해 해당 기술이나 제품의 우수성과 개선이 필요한 부분을 파악할 수 있습니다. 주로 실험, 시험, 분석 등의 방법을 사용하여 이루어지며, 기술의 개발 및 향상에 도움을 줍니다.


### 2. RAG구조와 vector DB를 활용한 질의

1) 참조할 데이터 로드 및 청킹

In [34]:
# 한 번에 여러개의 PDF로드
loaders = [PyPDFLoader("data/기술보증기금과 한국경제.pdf"),    # 5page
           PyPDFLoader("data/기술보증기금 주요업무.pdf")       # 4page
          ]

In [35]:
# 청킹을 위한 splitter 객체 생성
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=50,
                                              separators=["\n\n", "\n", " "]
                                             )

In [36]:
my_chunks = []   # 청크들을 보관할 리스트

for loader in loaders : 
    # 2개의 PDF파일에서 추출된 document객체들을 하나의 리스트에 합쳐줌
     # load_and_split함수를 사용하면 반환되는 형식이 리스트 이므로 extend 함수를 사용
    my_chunks.extend(loader.load_and_split(pdf_splitter))

print("청크 개수 :", len(my_chunks))
print()
my_chunks

청크 개수 : 13



[Document(metadata={'producer': 'Microsoft® Word Office 365용', 'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'moddate': '2019-06-27T22:46:56+09:00', 'source': 'data/기술보증기금과 한국경제.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하

#### 2) 임베딩 설정

In [10]:
# 임베딩 모델 API를 사용하여 임베딩 객체 생성
 # text-embedding-3-small 모델의 최대 입력 가능 토큰수는 8191개, 벡터의 길이(차원)는 1536차원
my_embedding = OpenAIEmbeddings(model='text-embedding-3-small',
                                api_key=MY_API_KEY
                               )
my_embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001DAC4F81E80>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001DAC4F82660>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [14]:
# 임베딩 예시
my_list = ["오늘은 비가오는 우중충한 날씨네요 허리조심하세요.", "두번째 문장은 여기까지"]

temp = my_embedding.embed_documents(my_list)
print("my_list 길이 :", len(temp))
print("첫 문장 길이 :", len(temp[0]))
print("두 번째 문장 길이 :", len(temp[1]))
print(temp)

my_list 길이 : 2
첫 문장 길이 : 1536
두 번째 문장 길이 : 1536
[[0.07731262594461441, -0.009142858907580376, -0.08033473789691925, 0.014115961268544197, 0.017740588635206223, -0.013876870274543762, -0.010386134497821331, 0.004640765953809023, -0.03551943227648735, -0.06610401719808578, -0.029609089717268944, 0.08905679732561111, -0.011658101342618465, -0.02090615965425968, 0.03305200859904289, 0.01641123928129673, -0.0009073521941900253, 0.016506876796483994, 0.0038756730500608683, 0.011753737926483154, -0.07157442718744278, -0.04142977297306061, 0.0028332341462373734, 0.03194262459874153, -0.00703407172113657, 0.016286911442875862, 0.00889898557215929, 0.020389722660183907, 0.0007979678339324892, -0.01669814996421337, 0.020160194486379623, -0.048162590712308884, 0.07321937382221222, -0.05822356045246124, 0.015282727777957916, -0.014536762610077858, 0.019739393144845963, -0.007846982218325138, -0.020944414660334587, -0.006838016677647829, -0.01798924431204796, -0.016602512449026108, 0.087144061923027

#### 3) Chorma DB 생성

In [15]:
# langchain연동 chroam DB 설치
!pip install langchain-chroma

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.6 MB ? eta -:--:--
   ------------ --------------------------- 6.3/19.6 MB 44.9 MB/s eta 0:00:01
   ----------------------------- ---------- 14.4/19.6 MB 33.8 MB/s eta 0:00:01
   -------------------------------------- - 18.9/19.6 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 19.6/19.6 MB 26.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.5 MB ? eta -:--:--
   ---------------------------------------- 4.5/4.5 MB 38.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 22.0 MB/s eta 0

In [40]:
# Chroma 클래스 임포트
from langchain_chroma import Chroma

In [41]:
my_directory = "/Vectorstores"

# from_documents : 청킹된 문서(documents객체)의 정보 및 임베딩, 경로 정보로 Chroma DB 생성
vectordb = Chroma.from_documents(documents=my_chunks,             # 벡터화 시킬 청크 목록
                                 embedding=my_embedding,          # 임베딩 객체
                                 persist_directory=my_directory   # 벡터 DB 저장 경로
                                )

#### <한 번 생성된 DB 불러와서 사용하기(쥬피터노트북 or 코랩을 껐다 켰을 경우)>
- 같은 경로에 DB를 계속 생성하면 중복된 데이터가 반복되어 저장됨
- 따라서 한 번 DB를 생성하고 데이터를 저장해뒀다면 다음 접속에서는 기존 DB를 불러와서 사용하는 방식으로 진행해야함

In [44]:
my_directory = "/VectorStores"

# 기존 DB의 내용을 불러올 때는 청크는 따로 지정할 필요가 없고, embedding_function으로 임베딩 객체를 지정
vectordb = Chroma(persist_directory=my_directory,
                  embedding_function=my_embedding
                 )

In [45]:
vectordb._collection.count()

13

#### Chroma객체로 사용할 수 있는 메소드 종류
- _collection.count : 수집된 벡터들의 개수 확인
- _collection.get : 수집된 벡터들의 정보 확인
- delete : 특정 id를 가진 벡터 삭제
- add_documents : 여러 문서를 벡터 DB에 추가
- add_texts : 여러 텍스트를 벡터 DB에 추가
- similarity_search : 유사도 계산을 통해 입력 질의에 대해 가장 유사한 문서(청크) 검색

(1) DB 정보 확인

In [42]:
# 수집된 문서(청크) 개수 확인
vectordb._collection.count()

13

In [43]:
# 수집된 청크들 id, documents(원본 문서), metadatas(문서와 관련된 정보)등을 확인
vectordb._collection.get()

{'ids': ['d85d313b-8a4a-4449-ab9f-1aaf8528c9eb',
  'c682e6b3-d8df-467e-9697-de5a0cf20966',
  '1fab6766-78ec-4fd7-8365-a5283e1fe756',
  '20d4d90a-5c3e-4d56-959e-030e8a4cee9e',
  '676b6776-f72b-4d02-9396-defea828ee0c',
  'e40c9ad4-4bb6-4ae6-b3d5-b5289887d1d3',
  '660a2459-fc2c-46d0-91ef-3dca687c4049',
  '501b2530-4b3c-46a7-8c37-75f2ef116ea5',
  '0b5cb8d7-9f53-48c3-931a-a48d3466990b',
  '2a0c2e2a-c34e-41e8-9c54-a1e571b51f7a',
  '1e8a763b-9ef5-4003-aa3d-34c52053c717',
  'ec8a412b-4cfb-4a5f-b489-6d1f4dfb501e',
  'd938e68b-18b9-4610-8aba-b5f8b30c803d'],
 'embeddings': None,
 'documents': ['페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인

(2) 기존 DB에 문서 추가하기

In [24]:
add_loaders = [PyPDFLoader("data/기술보증기금 주요업무_기술평가.pdf"),    # 5page
               PyPDFLoader("data/기술보증기금 주요업무_기술보증.pdf")       # 4page
               ]

add_chunks = []         # 추가할 청크 보관 리스트  

for loader in add_loaders : 
    add_chunks.extend(loader.load_and_split(pdf_splitter))

print("새로 추가할 청크 수 :", len(add_chunks))

새로 추가할 청크 수 : 15


In [25]:
# 벡터 DB에 문서 추가 (추가 후 추가된 각 벡터에 대한 id값 정보가 출력됨)
vectordb.add_documents(add_chunks)

['ae422daf-ecbd-4edb-bd6d-1076715a5faf',
 'ea597cf9-3529-4016-b69e-690b96e0f596',
 '7979a485-92ce-4362-9e63-62c89e0db665',
 '30aa7ecb-e715-4940-8b46-11c9b5e97e87',
 'defceaad-5057-4fae-a93d-37f611545ee1',
 '2eaadb06-2816-4d52-88c8-fc8c072c7811',
 '9a3c564f-9de0-45e4-b21a-eda67c8eb94a',
 'b90582a5-9fc8-43c2-8820-041b7df4a048',
 'cff82d3d-adfb-4f26-b27a-adab5e5feb6b',
 '690f22fa-1038-447d-9c94-d113c08e6ea2',
 '06c4e5f1-e16b-49e8-ab98-c208fc9930a9',
 '925e67d6-e387-424a-9d20-ccb89e0cf784',
 '49fb2b37-2c0c-4914-85aa-7a4602102cb1',
 '17ead2e9-5c2d-406f-81bb-1a3047d7ce0c',
 '5e0b6ded-715c-43f4-b0d9-c3a69d44f496']

In [39]:
vectordb._collection.count()
# 기존 13개의 청크에서 15개가 추가되어 총 28개

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 

(3) DB 값 삭제

In [27]:
vectordb._collection.get()

{'ids': ['2c6417ec-9359-46da-9d4f-621b719cc5ed',
  '68855d67-6c3f-4fe1-abd6-925fb9ff64ab',
  '04542ba6-5ac8-48f8-975d-c1a1a75ecdf6',
  '202b563d-1da3-4438-9c5b-c28f3a128170',
  '7f066e74-5be6-406d-944b-de8b2b99027d',
  '596c5dc7-0724-4b39-8645-b6f90fcb09b4',
  '876a8756-2fcd-489f-a5c5-336268afff6e',
  '596244c9-17bc-4b41-84ba-cc763b98d5a8',
  '96b2e835-d894-43ae-8d32-b12252968496',
  'ae878edd-6577-40af-89a1-8b294f15f501',
  'dcee2704-c0c0-49b9-b8ab-b88f6f221495',
  'f80178e3-48a1-4d78-b685-a398af4200bd',
  '91c33065-18b2-4af9-bd0a-66fdce3f9e59',
  'ae422daf-ecbd-4edb-bd6d-1076715a5faf',
  'ea597cf9-3529-4016-b69e-690b96e0f596',
  '7979a485-92ce-4362-9e63-62c89e0db665',
  '30aa7ecb-e715-4940-8b46-11c9b5e97e87',
  'defceaad-5057-4fae-a93d-37f611545ee1',
  '2eaadb06-2816-4d52-88c8-fc8c072c7811',
  '9a3c564f-9de0-45e4-b21a-eda67c8eb94a',
  'b90582a5-9fc8-43c2-8820-041b7df4a048',
  'cff82d3d-adfb-4f26-b27a-adab5e5feb6b',
  '690f22fa-1038-447d-9c94-d113c08e6ea2',
  '06c4e5f1-e16b-49e8-ab98-

In [31]:
# id값으로 벡터 삭제
vectordb.delete(ids=['68855d67-6c3f-4fe1-abd6-925fb9ff64ab',  '596c5dc7-0724-4b39-8645-b6f90fcb09b4' ])
vectordb._collection.count()

26

In [38]:
# 모든 데이터 삭제
vectordb.delete_collection()

In [33]:
vectordb._collection.count()
# 저장된 값이 없으면 에러 발생

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 

### 4) 유사도 검색
- 간단한 질문에 대해 유사한 문서 검색

In [46]:
question = "기술평가가 뭐야?"

# similarity_search : 유사도 검색 실행 함수
 # k : 반환 하고자 하는 document객체(청크) 수 지정
 #     (k값이 너무 크면 연관성이 떨어지는 청크까지 검색되어 LLM이 필요없는 내용도 참조해버릴 수 있음)
similar_docs = vectordb.similarity_search(question, k=3)
print("객체 수 : ", len(similar_docs))
print()
similar_docs

객체 수 :  3



[Document(id='676b6776-f72b-4d02-9396-defea828ee0c', metadata={'creator': 'Microsoft® Word Office 365용', 'page_label': '3', 'total_pages': 5, 'moddate': '2019-06-27T22:46:56+09:00', 'producer': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'page': 2, 'source': 'data/기술보증기금과 한국경제.pdf'}, page_content="페이지 3 / 5 \n \n신기술사업에 대한 자금의 공급을 원활하게 하고 나아가 국민경제의 발전에 이바지함 \n✓ 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 위하\n여 기술보증기금 설립 \n \n2. (참고) 기보 역할과 과제  \n(2019년3) 강조되는 key words \n- 국내 최초 기술평가시스템 도입→ (기술)중소기업의 미래가치 발굴+기술금융 신시장 개척  \n- 축적된 기술평가 역량 활용 → 기술혁신형 기업에 기술보증∙ 기술이전∙투자∙컨설팅 등 금융과 비\n금융 사업을 함께 지원함으로써--→기업 기술경쟁력↑→ 우리경제 신성장 동력(저성장공포 극복) 창출 \n- 환경 급변(예-성장둔화, 미-중 무역분쟁, 국가간 기술전쟁, 기술중소기업의 자금부족)에 대응 -→ 기술력 \n있는 중소 기업이 (99-88) 양질의 일자리 창출+한국 경제 혁신성장 주도해야 \n⚫ 기술 중소기업이 Scale-up4하도록 기술보증이 함께해야 할 것이다  \nMission5 \n중소∙벤처기업을 위한 기술금융과 혁신지원 활성화로 국민경제 발전에 기여 \n \n \n3. 강조 point  \n \n                                           \n3 2019.7월 현재 홈페이지 CEO 인사 중  \n4

### 5) MMR 유사도 검색
- 사용자 쿼리에 가장 관련성이 높은 문서를 선택하지만 중복된 정보를 줄이고, 다양한 정보를 제공하는 것을 목표로 하는 검색 방법

In [48]:
# fetch_k : 벡터 DB로 부터 가져올 유사도가 높은 문서 수
 # k : 응답의 다양성을 고려하여 fetch_k 중에서 반환할 문서 수(fetch_k에서 중복된 문서를 줄이고 다양성을 포함한 k개를 반환)
# lambda_mult : 0~! 사이 실수값으로 관련성과 다양성 사이의 균형을 제어(0에 가까울 수록 다양성 우선, 1에 가까울 수록 관련성 우선)
search_mmr = vectordb.max_marginal_relevance_search(question, fetch_k=6, k=3, lambda_mult=0.5)
search_mmr

# 일반적인 유사도 검색은 2,5,4page가 나왔는데 mmr검색은 2,4,6page가 나옴

[Document(id='676b6776-f72b-4d02-9396-defea828ee0c', metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'author': 'HS', 'total_pages': 5, 'creationdate': '2019-06-27T22:46:56+09:00', 'moddate': '2019-06-27T22:46:56+09:00', 'page': 2, 'creator': 'Microsoft® Word Office 365용', 'page_label': '3', 'producer': 'Microsoft® Word Office 365용'}, page_content="페이지 3 / 5 \n \n신기술사업에 대한 자금의 공급을 원활하게 하고 나아가 국민경제의 발전에 이바지함 \n✓ 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 위하\n여 기술보증기금 설립 \n \n2. (참고) 기보 역할과 과제  \n(2019년3) 강조되는 key words \n- 국내 최초 기술평가시스템 도입→ (기술)중소기업의 미래가치 발굴+기술금융 신시장 개척  \n- 축적된 기술평가 역량 활용 → 기술혁신형 기업에 기술보증∙ 기술이전∙투자∙컨설팅 등 금융과 비\n금융 사업을 함께 지원함으로써--→기업 기술경쟁력↑→ 우리경제 신성장 동력(저성장공포 극복) 창출 \n- 환경 급변(예-성장둔화, 미-중 무역분쟁, 국가간 기술전쟁, 기술중소기업의 자금부족)에 대응 -→ 기술력 \n있는 중소 기업이 (99-88) 양질의 일자리 창출+한국 경제 혁신성장 주도해야 \n⚫ 기술 중소기업이 Scale-up4하도록 기술보증이 함께해야 할 것이다  \nMission5 \n중소∙벤처기업을 위한 기술금융과 혁신지원 활성화로 국민경제 발전에 기여 \n \n \n3. 강조 point  \n \n                                           \n3 2019.7월 현재 홈페이지 CEO 인사 중  \n4

#### 6) as_retriever 클래스 사용
- as_retriever는 벡터DM 객체를 효율적인 검색기로 사용할 수 있도록 변환해주는 역할
- as_retriever를 사용하면 쿼리최적화 기법들이 적용되고 langchain생태계와의 연동성이 높아지며 유사도 검색 전후에 필터나 계약조건 등을 추가로 적용할 수 있어서 일반적인 검색기보다 더 성능이 좋음
- 추후 langchain의 RetrievalQA 클래스와 연동하여 질의-응답에 대한 전체적인 RAG chain을 구성할 수 있음

In [49]:
# as_retriever : 벡터 DB를 효율적인 검색기로 변환
 # search_kwargs={"k":3} : 검색할 유사도가 높은 청크 개수 지정(디폴트는 4개)
my_retriever = vectordb.as_retriever(search_kwargs={"k":3})

In [50]:
question = "기술평가가 뭐야?"

# invoke : 벡터 검색기에 질의를 입력하여 유사도가 높은 문서(청크) 반환
relevant_docs = my_retriever.invoke(question)
relevant_docs

[Document(id='676b6776-f72b-4d02-9396-defea828ee0c', metadata={'creator': 'Microsoft® Word Office 365용', 'creationdate': '2019-06-27T22:46:56+09:00', 'page': 2, 'total_pages': 5, 'moddate': '2019-06-27T22:46:56+09:00', 'page_label': '3', 'producer': 'Microsoft® Word Office 365용', 'source': 'data/기술보증기금과 한국경제.pdf', 'author': 'HS'}, page_content="페이지 3 / 5 \n \n신기술사업에 대한 자금의 공급을 원활하게 하고 나아가 국민경제의 발전에 이바지함 \n✓ 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 위하\n여 기술보증기금 설립 \n \n2. (참고) 기보 역할과 과제  \n(2019년3) 강조되는 key words \n- 국내 최초 기술평가시스템 도입→ (기술)중소기업의 미래가치 발굴+기술금융 신시장 개척  \n- 축적된 기술평가 역량 활용 → 기술혁신형 기업에 기술보증∙ 기술이전∙투자∙컨설팅 등 금융과 비\n금융 사업을 함께 지원함으로써--→기업 기술경쟁력↑→ 우리경제 신성장 동력(저성장공포 극복) 창출 \n- 환경 급변(예-성장둔화, 미-중 무역분쟁, 국가간 기술전쟁, 기술중소기업의 자금부족)에 대응 -→ 기술력 \n있는 중소 기업이 (99-88) 양질의 일자리 창출+한국 경제 혁신성장 주도해야 \n⚫ 기술 중소기업이 Scale-up4하도록 기술보증이 함께해야 할 것이다  \nMission5 \n중소∙벤처기업을 위한 기술금융과 혁신지원 활성화로 국민경제 발전에 기여 \n \n \n3. 강조 point  \n \n                                           \n3 2019.7월 현재 홈페이지 CEO 인사 중  \n4

In [51]:
# 검색기에 MMR검색도 적용 가능
my_retriever_mmr = vectordb.as_retriever(search_type="mmr",
                                         search_kwargs={"fetch_k":6, "k":3, "lambda_mult":0.5}
                                        )

In [52]:
my_retriever_mmr.invoke(question)

[Document(id='676b6776-f72b-4d02-9396-defea828ee0c', metadata={'page_label': '3', 'creator': 'Microsoft® Word Office 365용', 'source': 'data/기술보증기금과 한국경제.pdf', 'page': 2, 'moddate': '2019-06-27T22:46:56+09:00', 'author': 'HS', 'creationdate': '2019-06-27T22:46:56+09:00', 'producer': 'Microsoft® Word Office 365용', 'total_pages': 5}, page_content="페이지 3 / 5 \n \n신기술사업에 대한 자금의 공급을 원활하게 하고 나아가 국민경제의 발전에 이바지함 \n✓ 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 위하\n여 기술보증기금 설립 \n \n2. (참고) 기보 역할과 과제  \n(2019년3) 강조되는 key words \n- 국내 최초 기술평가시스템 도입→ (기술)중소기업의 미래가치 발굴+기술금융 신시장 개척  \n- 축적된 기술평가 역량 활용 → 기술혁신형 기업에 기술보증∙ 기술이전∙투자∙컨설팅 등 금융과 비\n금융 사업을 함께 지원함으로써--→기업 기술경쟁력↑→ 우리경제 신성장 동력(저성장공포 극복) 창출 \n- 환경 급변(예-성장둔화, 미-중 무역분쟁, 국가간 기술전쟁, 기술중소기업의 자금부족)에 대응 -→ 기술력 \n있는 중소 기업이 (99-88) 양질의 일자리 창출+한국 경제 혁신성장 주도해야 \n⚫ 기술 중소기업이 Scale-up4하도록 기술보증이 함께해야 할 것이다  \nMission5 \n중소∙벤처기업을 위한 기술금융과 혁신지원 활성화로 국민경제 발전에 기여 \n \n \n3. 강조 point  \n \n                                           \n3 2019.7월 현재 홈페이지 CEO 인사 중  \n4

- 표준 유사성 검색기는 유사도가 높은 문서 상위 k개를 빠르게 검색하므로 관련성이 높은 문서를 빠른 속도로 검색할 때 좋고
- mmr 유사성 검색기는 검색의 중복을 줄이고 다양성을 포함해야 할 때 사용하는 것이 좋음(다만 표준 검색기에 비해 약간 더 느림)

### 7) RetrievalQA 클래스 활용
- langchain에서 지원하는 질의응답 수행 클래스
- 위에서 만든 검색기 (my_retriever)와 LLM을 chain으로 묶어 통합하여 활용 가능

In [53]:
# RetrievalQA 내부에서 동작할 데이터 정의와 검증을 위한 라이브러리
!pip install pydantic

In [54]:
from langchain.chains import RetrievalQA
# 기존 openAI 자체에서 지원하는 completion객체는 입력 형태가 맞지 않아서 langchain과 연동된 OpenAI Chat모델을 활용
from langchain_openai import ChatOpenAI

In [58]:
chat_model = ChatOpenAI(model='gpt-3.5-turbo',
                        api_key=MY_API_KEY
                       )

In [59]:
R_QA1 = RetrievalQA.from_chain_type(llm=chat_model,              # LLM 지정
                                    chain_type = "refine",       # 답변을 위해 청크들이 결합되는 타입 지정
                                    retriever= my_retriever,     # 검색기 지정(my_retriever에는 우리가 만든 vector DE의 정보도 포함)
                                    return_source_documents=True # 어떤 내용을 참조했는지 출력 설정
                                   )

# <chain type 종류>
 # stuff : 검색된 청크들의 내용을 간단히 결합하여 답변 생성(작은 규모에 적합)
 # map_reduce : 청크들을 각각 처리한 후, 개별 결과를 결합하여 답변 생성(비교적 큰 규모에 적합)
 # refine : 이전 청크의 처리 결과를 다음 청크에 반영하여 지속적으로 응답을 개선한 후 최종 답변 생성
 # map_rerank : 각 청크를 개별 평가하여 관련성을 점수화한 후 가장 높은 문서를 선택하여 최종 답변 생성

In [60]:
question = "기술평가가 뭐야?"
llm_response = R_QA1.invoke(question)
llm_response

{'query': '기술평가가 뭐야?',
 'result': '기술평가는 기술의 가치나 수준을 측정하고 평가하는 과정을 말합니다. 이는 기술이 전문적으로 금액이나 다양한 방법을 통해 표현되어 하는 것을 포함합니다. 또한, 기술평가는 벤처 창업지원 및 벤처-이노비즈 인증 등을 통해 기술 우위를 바탕으로 경쟁력을 확보한 기술혁신형 중소기업을 지원하고, 기술경쟁력과 미래성장 가능성을 갖춘 기업들에 기술, 자금, 판로 등을 연계하여 지원하는 것으로 이해됩니다. 또한, 기술혁신형 기업을 위해 기술평가와 기술보증을 지원하고, 세계 최고수준의 기술평가시스템을 도입하여 재무제표 위주의 평가관행을 기술성과 사업성 위주로 변화시키는 역할을 수행하고 있습니다.',
 'source_documents': [Document(id='676b6776-f72b-4d02-9396-defea828ee0c', metadata={'page': 2, 'moddate': '2019-06-27T22:46:56+09:00', 'creator': 'Microsoft® Word Office 365용', 'author': 'HS', 'page_label': '3', 'creationdate': '2019-06-27T22:46:56+09:00', 'total_pages': 5, 'source': 'data/기술보증기금과 한국경제.pdf', 'producer': 'Microsoft® Word Office 365용'}, page_content="페이지 3 / 5 \n \n신기술사업에 대한 자금의 공급을 원활하게 하고 나아가 국민경제의 발전에 이바지함 \n✓ 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 위하\n여 기술보증기금 설립 \n \n2. (참고) 기보 역할과 과제  \n(2019년3) 강조되는 key words \n- 국내 최초 기술평가시스템 도입→ (기술)중소기업의 미래가치 발굴+기술금융 신시장 개척  \n- 축적된 기술평가 역량 활용 → 기술혁신형 기업에 기술보증∙

In [61]:
print(llm_response['query'])
print(llm_response['result'])

기술평가가 뭐야?
기술평가는 기술의 가치나 수준을 측정하고 평가하는 과정을 말합니다. 이는 기술이 전문적으로 금액이나 다양한 방법을 통해 표현되어 하는 것을 포함합니다. 또한, 기술평가는 벤처 창업지원 및 벤처-이노비즈 인증 등을 통해 기술 우위를 바탕으로 경쟁력을 확보한 기술혁신형 중소기업을 지원하고, 기술경쟁력과 미래성장 가능성을 갖춘 기업들에 기술, 자금, 판로 등을 연계하여 지원하는 것으로 이해됩니다. 또한, 기술혁신형 기업을 위해 기술평가와 기술보증을 지원하고, 세계 최고수준의 기술평가시스템을 도입하여 재무제표 위주의 평가관행을 기술성과 사업성 위주로 변화시키는 역할을 수행하고 있습니다.


#### RAG구조를 활용했을 때 소량의 데이터를 저장했음에도 일반적인 응답에 비해 훨씬 더 구체적이고 세부적인 답변이 나오는 것을 확인할 수 있음

### 8) Reranking(중요 문서 재판단) 사용
- reranking을 지웒나느 cohere라이브러리 사용(cohere 사에서 지원)
- 홈페이지 : https://dashboard.cohere.com/api-keys 에서 key발급 필요(가입하면 디폴트로 주어짐)
- CohereRerank 클래스
  - langchain과 cohere를 연동해 reranking을 지원하는 클래스

In [62]:
# langchain과 연동된 cohere 라이브러리 설치
!pip install langchain-cohere


  Attempting uninstall: httpx-sse

    Found existing installation: httpx-sse 0.4.1

   ------------- -------------------------- 2/6 [httpx-sse]
    Uninstalling httpx-sse-0.4.1:
   ------------- -------------------------- 2/6 [httpx-sse]
      Successfully uninstalled httpx-sse-0.4.1
   ------------- -------------------------- 2/6 [httpx-sse]
   ------------- -------------------------- 2/6 [httpx-sse]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   ----

In [63]:
from langchain_cohere import CohereRerank

In [64]:
COHERE_API_KEY = getpass.getpass("Cohere API Key :")

Cohere API Key : ········


In [65]:
# CohereRerank객체 생성
my_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY,
                         model="rerank-v3.5",   # cohere에서 지원하는 reranking모델
                         top_n=3                # rerank모델이 가장 관련성이 높은 3개의 결과만 반환
                        )

### 9) Contextual Compression (문맥 압축) 사용
- ContextualCompressionRetriever클래스
  - 검색된 문서의 문맥을 압축하여 보다 집중적이고 관련성 높은 정보를 제공하는 압축검색기(Rerank객체와 같이 사용 가능)
  - 다른 기능 및 검색기와 통합하여 사용할 수 있으며, base압축기에 의해 검색된 내용의 문맥을 압축하게 됨
  - **문맥 압축** : 가장 관련성이 높은 부분을 선택하고, 덜 중요한 정보는 삭제하여 핵심만 남겨두는 것.

In [66]:
from langchain.retrievers import ContextualCompressionRetriever

In [67]:
# 압축검색기 객체 설정
 # base_compressor에는 LLM기반 문서압축기 혹은 문서요약기, 임베딩 유사도 기반 필터기, rerank 등의 객체가 들어갈 수 있음
  # 1) 임베딩 유사도 기반 필터기, LLM 기반 문서 요약기 -> 빠른 속도 및 비용 절감에 유리
  # 2) LLM기반 문서 압축기, reranker -> 정확한 결과와 성능 확보에 유리
compression_retriever = ContextualCompressionRetriever(base_compressor=my_rerank,
                                                       base_retriever=my_retriever
                                                      )

In [69]:
R_QA2 = RetrievalQA.from_chain_type(llm=chat_model,             
                                    chain_type = "refine",    
                                    retriever= compression_retriever   # 압축검색기 지정
                                   )
llm_response2 = R_QA2.invoke(question)
llm_response2

{'query': '기술평가가 뭐야?',
 'result': '기술평가는 기술의 가치를 평가하고 가이드하는 과정으로, 기술혁신형 기업의 기술경쟁력을 높이고 성장동력을 창출하는 데 중요한 역할을 합니다. 기술평가는 기술성, 시장성, 사업타당성 등을 종합적으로 평가하여 기업의 기술적 역량을 정확하게 파악함으로써, 기술보증기금을 통해 중소기업의 기술금융 지원을 강화하고 기술혁신을 촉진하는 데 도움을 줄 수 있습니다. 또한, 환경의 급변에 대비하여 중소기업이 양질의 일자리를 창출하고 한국 경제의 혁신성장을 주도할 수 있도록 기업의 기술경쟁력을 높이는 데 기여할 수 있습니다.특히, 기술평가 시스템을 통해 중소기업의 미래가치 발굴과 기술금융 신시장 개척을 지원하며, 기술혁신형 기업에 다양한 금융과 비금융 지원을 제공하여 우리경제의 신성장 동력을 창출할 수 있습니다.'}